In [1]:
import numpy as np

In [2]:
np.random.randint(0,10)

1

In [3]:
import gym
from replay_buffer2 import ReplayBuffer 
import numpy as np
import time

In [2]:
gym.__file__

'/home/programmer/anaconda3/envs/rl/lib/python3.6/site-packages/gym/__init__.py'

In [4]:
env = gym.make("Taxi-v2")
env = gym.make("Blackjack-v0")

In [41]:
Q = np.zeros([1000,1])


In [42]:
Q[8]

array([0.])

In [38]:
env.action_space.sample()


1

In [7]:
env.observation_space

Tuple(Discrete(32), Discrete(11), Discrete(2))

In [4]:
print("State space ", env.observation_space)
print("Action space ", env.action_space.n)

State space  Discrete(500)
Action space  6


In [5]:
memory = ReplayBuffer((1,),(6,),int(1e5),'cpu')

In [103]:
class Agent():
    def __init__(self, state_size, action_size, lr, decay):
        self.action_size = action_size
        self.state_size = state_size
        self.Q = np.zeros([state_size, action_size])
        self.Q_shigt = np.zeros([state_size, action_size])
        self.r = np.zeros([state_size, action_size])
        self.counter = np.zeros([state_size, action_size])
        self.gamma = 0.618
        self.epsilon = 1
        self.lr = lr
        self.min_epsilon =0.01
        self.max_epsilon =1
        self.batch_size = 64
        self.episode = 20000
        self.decay = decay
        self.total_reward = 0
        self.start_learn = 1000
        self.eval_frq = 50
        self.render_env = False
        self.env = gym.make("Taxi-v2")
        self.memory = ReplayBuffer((1,),(1,),int(1e5),'cpu')
        self.gamma_iql = 0.99
        self.lr_sh = 0.005
        self.ratio = 1./ action_size
        self.eval_q_inverse = 1e4
        self.episodes_qinverse = int(1e5)
    
    def act(self, state, epsilon, eval_pi = False):
        if np.random.random() > epsilon or eval_pi:
            action = np.argmax(self.Q[state])
        else:
            action = env.action_space.sample() 
        return action
    
    
    def optimize(self, state, action, reward, next_state):
        max_next_state = np.max(self.Q[next_state])
        td_error =  max_next_state - self.Q[state, action]
        self.Q[(state,action)] = self.Q[(state,action)] + self.lr * (reward + self.gamma *td_error)
    
    def learn(self):
        states, actions, rewards, next_states, done =  self.memory.sample(self.batch_size)
        # update Q function
        
        for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, done):
            max_next_state = np.max(self.Q[next_state])
            td_error = self.Q[state, action] - max_next_state
            self.Q[(state,action)] = self.Q[(state,action)] + self.lr * (reward + self.gamma*  td_error)
    
    
    
    
    def invers_q(self, continue_train=False):
        if not continue_train:
            self.Q = np.zeros([self.state_size, self.action_size])
        for i in range(self.episodes_qinverse):
            if i == self.eval_q_inverse:
                self.eval_policy()
            state, action, _, next_state, _ = self.memory.sample(1)
            self.counter[state,action] +=1
            total_num = np.sum(self.counter[state,:])
            action_prob = self.counter[state] / total_num
            #print(np.sum(action_prob))
            assert(np.isclose(np.sum(action_prob),1))
            target = self.lr_sh *(self.gamma_iql * np.max(self.Q[next_state]))
            agent.Q_shigt[state, action] = (1 - self.lr_sh) * self.Q_shigt[state, action] + target
            n_a = np.zeros([self.action_size])
            for a in range(self.action_size):
                n_a[a] = action_prob[0][0][a] - self.Q_shigt[state,a]
            
            # update reward function
            self.update_r(state, action, n_a)
            # update Q function
            self.update_q(state, action, next_state)
    
    def update_q(self, state, action, next_state):
        q_old = (1 - self.lr) * self.Q[state, action]
        q_new = self.lr *(self.r[state, action] + self.gamma_iql * np.max(self.Q[next_state]))
        self.Q[state, action] = q_old + q_new
        
    def update_r(self, state, action, n_a):
        r_old = (1 - self.lr) * self.r[state, action]
        part1 = self.compute_n_a(action, action_prob)
        part2 = (self.ratio * self.sum_over_action(state, action, action_prob))
        r_new = self.lr * (part1 + part2)
        self.r[state, action] = r_old + r_new       
    
    def sum_over_action(self, state, a, action_prob):
        res = 0
        for b in range(self.action_size):
            if b == a:
                continue
            res += self.r[state, b] - self.compute_n_a(b, action_prob)
        return res
    
    def compute_n_a(self, a, action_prob):
        return action_prob[0][0][a] - self.Q_shigt[state,a]
    
    def eval_policy(self, random_agent=False, episode=10):
        total_steps = 0
        total_reward = 0
        total_penetlies = 0
        for i_episode in range(1, episode):
            score = 0
            steps = 0
            state = env.reset()
            done  = False
            penelty = 0
            while not done:
                steps += 1
                action = self.act(state, self.epsilon, True)
                if random_agent:
                    action = self.env.action_space.sample() 
                
                next_state, reward, done, _ = self.env.step(action)
                if self.render_env:
                    self.env.render()
                    time.sleep(0.1)
                score += reward
                if reward == -10:
                    penelty += 1
                if done or steps == 100:
                    total_steps += steps
                    total_reward += score
                    total_penetlies += penelty
                    break
        if self.render_env:
            self.env.close()
        aver_steps = total_steps / episode
        average_reward = total_reward / episode
        aver_penelties = total_penetlies / episode
        print("Eval avge steps {} average reward  {}  average penelty {} ".format(aver_steps, average_reward, aver_penelties))
        
    
    def train(self):
      
        total_timestep = 0
        for i_episode in range(1, self.episode + 1):
            score = 0
            state = self.env.reset()
            done  = False
            steps = 0
            while not done:
                steps += 1
                total_timestep += 1
                action = self.act(state, self.epsilon)
                next_state, reward, done, _ = self.env.step(action)
                score += reward
                self.optimize(state, action, reward, next_state)
                self.epsilon = self.min_epsilon + (self.max_epsilon - self.min_epsilon)*np.exp(-self.decay* i_episode)
                self.memory.add(state, action, reward, next_state, done, done)
                
                if done:
                    break
                state = next_state
            
            if i_episode % self.eval_frq == 0:
                self.eval_policy()
            
            self.total_reward += score
            average_reward = self.total_reward / i_episode
            print("Episode {} Reward {} Average Reward {} steps {}  epsilon {}".format(i_episode, score, average_reward, steps, self.epsilon))
            
        
        
        
        

In [104]:
env = gym.make("Taxi-v2")
state_space = env.observation_space.n
action_space = env.action_space.n
print("State space ", state_space)
print("Action space ", action_space)
memory = ReplayBuffer((1,),(6,),int(1e5),'cpu')

State space  500
Action space  6


In [105]:
lr = 0.7
decay = 0.01
agent = Agent(state_space, action_space, lr, decay)
agent.episode = 100
agent.train()

Episode 1 Reward -262 Average Reward -262.0 steps 100  epsilon 0.9901493354116764
Episode 2 Reward -370 Average Reward -316.0 steps 100  epsilon 0.9803966865736877
Episode 3 Reward -469 Average Reward -367.0 steps 100  epsilon 0.970741078213023
Episode 4 Reward -334 Average Reward -358.75 steps 100  epsilon 0.9611815447608
Episode 5 Reward -397 Average Reward -366.4 steps 100  epsilon 0.9517171302557069
Episode 6 Reward -397 Average Reward -371.5 steps 100  epsilon 0.9423468882484062
Episode 7 Reward -397 Average Reward -375.14285714285717 steps 100  epsilon 0.9330698817068888
Episode 8 Reward -397 Average Reward -377.875 steps 100  epsilon 0.9238851829227694
Episode 9 Reward -370 Average Reward -377.0 steps 100  epsilon 0.9147918734185159
Episode 10 Reward -334 Average Reward -372.7 steps 100  epsilon 0.9057890438555999
Episode 11 Reward -388 Average Reward -374.09090909090907 steps 100  epsilon 0.896875793943563
Episode 12 Reward -379 Average Reward -374.5 steps 100  epsilon 0.888051

In [106]:
agent.invers_q()

Eval avge steps 0.9 average reward  -0.9  average penelty 0.0 


In [93]:
agent.eval_policy(), agent.eval_policy(True)

Eval avge steps 0.9 average reward  -0.9  average penelty 0.0 
Eval avge steps 0.9 average reward  -0.9  average penelty 0.0 


(None, None)

In [34]:
episodes = 10000
for i in range(episodes):
    state, action, _, next_state, _ = agent.memory.sample(1)
    agent.counter[state,action] +=1
    total_num = np.sum(agent.counter[state,:])
    action_prob = agent.counter[state] / total_num
    #print(np.sum(action_prob))
    assert(np.isclose(np.sum(action_prob),1))
    target = agent.lr_sh *(agent.gamma_iql * np.max(agent.Q[next_state]))
    agent.Q_shigt[state, action] = (1 - agent.lr_sh) * agent.Q_shigt[state, action] + target
    n_a = np.zeros([agent.action_size])
    for a in range(agent.action_size):
        n_a[a] = action_prob[0][0][a] - agent.Q_shigt[state, a]
    

    
    # update reward function
    #print("main action", action)
    update_r(state, action, n_a, agent)
    # update Q function
    update_q(state, action, next_state, agent)

    
       
  

main action [[0]]
action a  [[0]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[2]]
action a  [[2]]
main action [[1]]
action a  [[1]]
main action [[3]]
action a  [[3]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[4]]
action a  [[4]]
main action [[2]]
action a  [[2]]
main action [[0]]
action a  [[0]]
main action [[5]]
action a  [[5]]
main action [[2]]
action a  [[2]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[4]]
action a  [[4]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[4]]
action a  [[4]]
main action [[1]]
action a  [[1]]
main action [[3]]
action a  [[3]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[5]]
action a  [[5]]
main action [[2]]
action a  [[2]]
main action [[0]]
action a  [[0]]
main action [[0]]
action a  [[0]]
main action [[0]]
action a  [[0]]
main action [[

main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[4]]
action a  [[4]]
main action [[3]]
action a  [[3]]
main action [[3]]
action a  [[3]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[2]]
action a  [[2]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[3]]
action a  [[3]]
main action [[1]]
action a  [[1]]
main action [[2]]
action a  [[2]]
main action [[3]]
action a  [[3]]
main action [[5]]
action a  [[5]]
main action [[0]]
action a  [[0]]
main action [[0]]
action a  [[0]]
main action [[2]]
action a  [[2]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[4]]
action a  [[4]]
main action [[

action a  [[0]]
main action [[5]]
action a  [[5]]
main action [[2]]
action a  [[2]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[4]]
action a  [[4]]
main action [[4]]
action a  [[4]]
main action [[2]]
action a  [[2]]
main action [[1]]
action a  [[1]]
main action [[3]]
action a  [[3]]
main action [[0]]
action a  [[0]]
main action [[2]]
action a  [[2]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[2]]
action a  [[2]]
main action [[4]]
action a  [[4]]
main action [[4]]
action a  [[4]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[5]]
action a  [[5]]
main action [[3]]
action a  [[3]]
main action [[2]]
action a  [[2]]
main action [[1]]
action a  [[1]]
main action [[5]]
action a  [[5]]
main action [[5]]
action a  [[5]]
main action [[1]]
action a  [[1]]
main action [[2]]
action a  [[2]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]

main action [[0]]
action a  [[0]]
main action [[3]]
action a  [[3]]
main action [[3]]
action a  [[3]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[0]]
action a  [[0]]
main action [[3]]
action a  [[3]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[4]]
action a  [[4]]
main action [[3]]
action a  [[3]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[5]]
action a  [[5]]
main action [[0]]
action a  [[0]]
main action [[2]]
action a  [[2]]
main action [[3]]
action a  [[3]]
main action [[3]]
action a  [[3]]
main action [[0]]
action a  [[0]]
main action [[3]]
action a  [[3]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[

action a  [[4]]
main action [[2]]
action a  [[2]]
main action [[0]]
action a  [[0]]
main action [[3]]
action a  [[3]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[2]]
action a  [[2]]
main action [[3]]
action a  [[3]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[4]]
action a  [[4]]
main action [[3]]
action a  [[3]]
main action [[3]]
action a  [[3]]
main action [[2]]
action a  [[2]]
main action [[0]]
action a  [[0]]
main action [[0]]
action a  [[0]]
main action [[2]]
action a  [[2]]
main action [[0]]
action a  [[0]]
main action [[0]]
action a  [[0]]
main action [[3]]
action a  [[3]]
main action [[2]]
action a  [[2]]
main action [[5]]
action a  [[5]]
main action [[0]]
action a  [[0]]
main action [[0]]
action a  [[0]

...

main action [[1]]
action a  [[1]]
main action [[2]]
action a  [[2]]
main action [[0]]
action a  [[0]]
main action [[0]]
action a  [[0]]
main action [[4]]
action a  [[4]]
main action [[4]]
action a  [[4]]
main action [[3]]
action a  [[3]]
main action [[2]]
action a  [[2]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[5]]
action a  [[5]]
main action [[3]]
action a  [[3]]
main action [[2]]
action a  [[2]]
main action [[0]]
action a  [[0]]
main action [[2]]
action a  [[2]]
main action [[0]]
action a  [[0]]
main action [[5]]
action a  [[5]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[4]]
action a  [[4]]
main action [[3]]
action a  [[3]]
main action [[1]]
action a  [[1]]
main action [[2]]
action a  [[2]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[4]]
action a  [[4]]
main action [[

action a  [[5]]
main action [[0]]
action a  [[0]]
main action [[0]]
action a  [[0]]
main action [[0]]
action a  [[0]]
main action [[3]]
action a  [[3]]
main action [[4]]
action a  [[4]]
main action [[2]]
action a  [[2]]
main action [[3]]
action a  [[3]]
main action [[0]]
action a  [[0]]
main action [[2]]
action a  [[2]]
main action [[2]]
action a  [[2]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[0]]
action a  [[0]]
main action [[5]]
action a  [[5]]
main action [[0]]
action a  [[0]]
main action [[3]]
action a  [[3]]
main action [[5]]
action a  [[5]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[2]]
action a  [[2]]
main action [[3]]
action a  [[3]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[2]]
action a  [[2]]
main action [[3]]
action a  [[3]]
main action [[3]]
action a  [[3]]
main action [[0]]
action a  [[0]

main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[5]]
action a  [[5]]
main action [[0]]
action a  [[0]]
main action [[2]]
action a  [[2]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[2]]
action a  [[2]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[5]]
action a  [[5]]
main action [[3]]
action a  [[3]]
main action [[2]]
action a  [[2]]
main action [[2]]
action a  [[2]]
main action [[2]]
action a  [[2]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[3]]
action a  [[3]]
main action [[1]]
action a  [[1]]
main action [[2]]
action a  [[2]]
main action [[1]]
action a  [[1]]
main action [[3]]
action a  [[3]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[

main action [[2]]
action a  [[2]]
main action [[0]]
action a  [[0]]
main action [[4]]
action a  [[4]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[3]]
action a  [[3]]
main action [[2]]
action a  [[2]]
main action [[3]]
action a  [[3]]
main action [[3]]
action a  [[3]]
main action [[2]]
action a  [[2]]
main action [[3]]
action a  [[3]]
main action [[5]]
action a  [[5]]
main action [[5]]
action a  [[5]]
main action [[1]]
action a  [[1]]
main action [[4]]
action a  [[4]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[5]]
action a  [[5]]
main action [[1]]
action a  [[1]]
main action [[2]]
action a  [[2]]
main action [[0]]
action a  [[0]]
main action [[5]]
action a  [[5]]
main action [[1]]
action a  [[1]]
main action [[5]]
action a  [[5]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[1]]
action a  [[1]]
main action [[

main action [[0]]
action a  [[0]]
main action [[4]]
action a  [[4]]
main action [[5]]
action a  [[5]]
main action [[1]]
action a  [[1]]
main action [[2]]
action a  [[2]]
main action [[3]]
action a  [[3]]
main action [[2]]
action a  [[2]]
main action [[5]]
action a  [[5]]
main action [[2]]
action a  [[2]]
main action [[3]]
action a  [[3]]
main action [[1]]
action a  [[1]]
main action [[2]]
action a  [[2]]
main action [[0]]
action a  [[0]]
main action [[3]]
action a  [[3]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[0]]
action a  [[0]]
main action [[5]]
action a  [[5]]
main action [[5]]
action a  [[5]]
main action [[4]]
action a  [[4]]
main action [[0]]
action a  [[0]]
main action [[2]]
action a  [[2]]
main action [[0]]
action a  [[0]]
main action [[2]]
action a  [[2]]
main action [[2]]
action a  [[2]]
main action [[0]]
action a  [[0]]
main action [[1]]
action a  [[1]]
main action [[2]]
action a  [[2]]
main action [[3]]
action a  [[3]]
main action [[

In [50]:
state = env.reset()
ns, reward, done, _ = env.step(5)
reward

-10

In [45]:
agent.r[state,0], agent.r[state,1], agent.r[state,2], agent.r[state,3], agent.r[state,4], agent.r[state,5]

(0.0, 0.0015742208091720049, 0.0, 0.10743689724975072, 0.0, 0.0)

In [26]:
state, action, _, next_state, _ = agent.memory.sample(1)
action

array([[0]], dtype=int8)

In [31]:
def update_q(state, action, next_state, agent):
    q_old = (1 - agent.lr) * agent.Q[state, action]
    q_new = agent.lr *(agent.r[state, action] + agent.gamma_iql * np.max(agent.Q[next_state]))
    agent.Q[state, action] = q_old + q_new
        

In [30]:
def update_r(state, action, n_a, agent):
    r_old = (1 - agent.lr) * agent.r[state, action]
    part1 = compute_n_a(action, action_prob, agent)
    part2 = (agent.ratio * sum_over_action(state, action, action_prob, agent))
    r_new = agent.lr * (part1  + part2)
    agent.r[state, action] = r_old + r_new 

In [28]:
def sum_over_action(state, a, action_prob, agent):
    print("action a ", a)
    res = 0
    for b in range(agent.action_size):
        if b == a:
            continue
        res += agent.r[state, b] - compute_n_a(b, action_prob, agent)
    return res

In [29]:
def compute_n_a( a, action_prob, agent):
    return action_prob[0][0][a] - agent.Q_shigt[state,a]

In [65]:
agent.eval_policy()

Eval avge steps 0.9 average reward  -0.9  average penelty 0.0 


In [86]:
agent.invers_q()

[0.         0.         0.         0.         0.95194175 0.        ]


In [66]:
agent.gamma_iql = 0.99
agent.lr_sh = 0.5

In [79]:
episodes = 10000
for i in range(episodes):
    state, action, _, next_state, _ = agent.memory.sample(1)
    agent.counter[state,action] +=1
    total_num = np.sum(agent.counter[state,:])
    
    action_prob = agent.counter[state] / total_num
    print(action_prob[0][0])
    #print(np.sum(action_prob))
    assert(np.isclose(np.sum(action_prob),1))
    target = agent.lr_sh *(agent.gamma_iql * np.max(agent.Q[next_state]))
    agent.Q_shigt[state, action] = (1 - agent.lr_sh) * agent.Q_shigt[state, action] + target
    n_a = np.zeros([agent.action_size])
    for a in range(agent.action_size):
        n_a[a] = action_prob[0][0][a] - agent.Q_shigt[state,a]
    print(n_a)
    break

[0.99113082 0.00665188 0.00221729 0.         0.         0.        ]
[-9.42149054e+00  6.65188470e-03  2.21729490e-03  0.00000000e+00
  0.00000000e+00  0.00000000e+00]


In [56]:
agent.counter

array([[  1.,   0.,   0., 159.,   0.,   0.],
       [  1.,   0., 136.,   0.,  48.,   0.],
       [  0.,   1.,   0., 142.,  60.,   0.],
       ...,
       [  0.,   0.,  25.,   0.,   0.,   0.],
       [  0.,   0.,  34.,   1.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.]])

In [34]:
agent.render_env= True
agent.eval_policy()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
Eval avge steps 0.9 average reward  -0.9  average penelty 0.0 


In [16]:
agent.eval_policy()

Eval avge steps 90.0 average reward  -90.0  average penelty 0.0 


In [69]:
memory = agent.memory

In [70]:
memory

88706